In [1]:
# Install required packages (run once in Colab)
!pip install qiskit qiskit-aer qiskit-nature qiskit-algorithms pyscf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00


In [2]:
# Imports
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimatorV2
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import n_local

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [3]:
# 1. Define the molecule (H2 at bond length 0.735 Å)
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto3g")
driver_result = driver.run()  # run driver to get molecular integrals

# 2. Get second-quantized Hamiltonian
second_q_ops = driver_result.second_q_ops()
hamiltonian = second_q_ops[0]

# 3. Map Fermionic Hamiltonian to Qubits using Jordan-Wigner
mapper = JordanWignerMapper()
qubit_op = mapper.map(hamiltonian)
num_qubits = qubit_op.num_qubits

# 4. Define ansatz (trial wavefunction) using n_local
ansatz = n_local(
    num_qubits=num_qubits,
    rotation_blocks="ry",
    entanglement_blocks="cz",
    reps=2
)

# 5. Set optimizer
optimizer = SLSQP(maxiter=100)

# 6. Set backend and estimator
simulator = AerSimulator(method="statevector")
estimator = BackendEstimatorV2(backend=simulator)

# 7. Initialize VQE
vqe = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)

# 8. Run VQE to compute ground state energy
result = vqe.compute_minimum_eigenvalue(qubit_op)

# 9. Print results
print("Ground state energy of H2 (Hartree):", result.eigenvalue.real)

Ground state energy of H2 (Hartree): -0.9309173710122725
